### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv('.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f'{url}&api-key={nyt_api_key}&sort={sort}&facet_fields=source&facet=true&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&fl={field_list}'

In [ ]:
# Create an empty list to store the reviews
reviews_list =[]

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url = f"{query_url}&page={str(page)}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in range(len(reviews["response"]["docs"])):
            reviews_list.append(reviews["response"]["docs"][review])
        # Print the page that was just retrieved
        print(f'Checked page {page}')
        # Print the page number that had no results then break from the loop
    except:
        print(f'Page {page} had no results')

Checked page 0
Checked page 1
Checked page 2


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list,indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
NYT_df = pd.json_normalize(reviews_list)
NYT_df

In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
NYT_df['title'] = NYT_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
NYT_df

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
NYT_df['keywords'] = NYT_df['keywords'].apply(extract_keywords)
NYT_df

In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = NYT_df['title'].to_list()
titles

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter%50 == 0:
        time.sleep(1)
        print('The application is sleeping')
    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    response_data = requests.get(url+title+tmdb_key_string).json()

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Get movie id
        movie_id = response_data["results"][0]["id"]

        # Make a request for a the full movie details
        details = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}').json()

        # Execute "GET" request with url

        
        # Extract the genre names into a list
        genres = []
        for genre in range(len(details['genres'])):
            genres.append(details['genres'][genre]['name'])
            
        # Extract the spoken_languages' English name into a list
        spoken_languages = []
        for language in range(len(details['spoken_languages'])):
            spoken_languages.append(details['spoken_languages'][language]['english_name'])

        # Extract the production_countries' name into a list
        production_countries = []
        for country in range(len(details['production_countries'])):
            production_countries.append(details['production_countries'][country]['name'])

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        dictionary ={
            'title': details['title'],
            'original_title': details['original_title'],
            'budget': details['budget'],
            'genre': genres,
            'language': details['original_language'],
            'spoken_languages':spoken_languages,
            'homepage': details['homepage'],
            'overview': details['overview'],
            'popularity': details['popularity'],
            'runtime': details['runtime'],
            'revenue': details['revenue'],
            'release_date': details['release_date'],
            'vote_average': details['vote_average'],
            'vote_count': details['vote_count'],
            'production_countries': production_countries
        }
        tmdb_movies_list.append(dictionary)
        # Print out the title that was found
        print(f'Found {title}')
    except:
        print(f'{title} not found.')

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list,indent=4)).head()

In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(tmdb_df,NYT_df,on='title')
merged_df

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove = ['[',']',"'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column]= merged_df[column].astype('str')

    # Loop through characters to remove
    for character in characters_to_remove:
        merged_df[column]=merged_df[column].str.replace(pat=character,repl='')

# Display the fixed DataFrame
merged_df.head()

In [ ]:
# Drop "byline.person" column
merged_df=merged_df.drop(columns="byline.person")

In [ ]:
# Delete duplicate rows and reset index
merged_df=merged_df.drop_duplicates()
merged_df=merged_df.reset_index(drop=True)
merged_df

In [ ]:
# Export data to CSV without the index
merged_df.to_csv("result", encoding="utf-8", index=False)